In [ ]:
# GSA with SOBOL

In [1]:
import time
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('white')
# import statsmodels.api as sm

from ema_workbench import (Model, RealParameter, IntegerParameter, ArrayOutcome, TimeSeriesOutcome,
                           ema_logging, SequentialEvaluator,
                           MultiprocessingEvaluator)
from ema_workbench import save_results, load_results
# from ema_workbench.analysis import prim
from ema_workbench.analysis import scenario_discovery_util as sdutil
from sklearn import preprocessing 
from ema_workbench.em_framework.evaluators import LHS, SOBOL, MORRIS
from ema_workbench.analysis import feature_scoring
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from SALib.analyze import sobol




ema_logging.log_to_stderr(ema_logging.INFO)
from dicemodel.noDICE_v2 import PyDICE

C:\Users\shrid\Anaconda3\lib\site-packages\ema_workbench\analysis\prim.py:31: ImportWarning: altair based interactive inspection not available
  "inspection not available"), ImportWarning)
C:\Users\shrid\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


In [2]:
# from openExp_working import dice_sm.uncertainties, dice_sm.levers, dice_sm.outcomes

In [3]:

if __name__ == '__main__':
    ema_logging.log_to_stderr(ema_logging.INFO)
    

    model = PyDICE()
    dice_sm = Model('dicesmEMA', function=model)
    
    dice_sm.uncertainties = [IntegerParameter('t2xco2_index', 0, 999),
                             IntegerParameter('t2xco2_dist',0,2),
                             IntegerParameter('fdamage', 0, 2),
                             RealParameter('tfp_gr',  0.07, 0.09),
                             RealParameter('sigma_gr', -0.012, -0.008),
                             RealParameter('pop_gr', 0.1, 0.15),
                             RealParameter('emdd',  0.01, 2.00),
                             RealParameter('fosslim',  4000.0, 13649),
                             IntegerParameter('cback', 100, 600)]
    
    dice_sm.levers = [RealParameter('sr', 0.1, 0.5),
                      RealParameter('prtp_con',  0.001, 0.015),
                      RealParameter('prtp_dam',  0.001, 0.015),
                    #   RealParameter('emuc',  0.01, 2.00),                      
                      IntegerParameter('periodfullpart', 10, 58),
                      IntegerParameter('miu_period', 10, 58)]
    
    dice_sm.outcomes = [TimeSeriesOutcome('Atmospheric Temperature'),
                        TimeSeriesOutcome('Damages'),
                        TimeSeriesOutcome('Utility of Consumption'),
                        TimeSeriesOutcome('Disutility of Damage'),
                        # TimeSeriesOutcome('Damage to output ratio'),
                        TimeSeriesOutcome('Welfare'),
                        TimeSeriesOutcome('Total Output'),
                        TimeSeriesOutcome('Consumption SDR'),
                        TimeSeriesOutcome('Damage SDR')
                        ]
    

In [4]:
problem = get_SALib_problem(dice_sm.uncertainties)
problem

{'num_vars': 9,
 'names': ['cback',
  'emdd',
  'fdamage',
  'fosslim',
  'pop_gr',
  'sigma_gr',
  't2xco2_dist',
  't2xco2_index',
  'tfp_gr'],
 'bounds': [(100, 601),
  (0.01, 2.0),
  (0, 3),
  (4000.0, 13649),
  (0.1, 0.15),
  (-0.012, -0.008),
  (0, 3),
  (0, 1000),
  (0.07, 0.09)]}

In [6]:
n_scenarios = 100
n_policies = 50

In [7]:
start = time.time()
with MultiprocessingEvaluator(dice_sm) as evaluator:
       results_sobol = evaluator.perform_experiments(n_scenarios, n_policies, uncertainty_sampling=SOBOL)
end = time.time()

print('Experiment time is ' + str(round((end - start)/60)) + ' mintues')
# Num of runs needed for k uncertainties = 1000(k+2)
# for 9 uncertainties, 11,000 runs
# with 100 scenarios and 10 policies under sobol, we get 18000 experiments/ runs (now 20,000: how does this work?)
#  1000s, 100p = 2million experiments. starting time 1:13 am

[MainProcess/INFO] pool started
[MainProcess/INFO] performing 2000 scenarios * 50 policies * 1 model(s) = 100000 experiments
[MainProcess/INFO] 10000 cases completed
[MainProcess/INFO] 20000 cases completed
[MainProcess/INFO] 30000 cases completed
[MainProcess/INFO] 40000 cases completed
[MainProcess/INFO] 50000 cases completed
[MainProcess/INFO] 60000 cases completed


In [ ]:
save_results(results_sobol, '2a_SOBOL_100s_50p_100000e_fdamage.tar.gz')
#results = load_results('SOBOL_160s_10p.tar.gz')

In [ ]:
# print(results_sobol)
experiments, outcomes = results_sobol
# outcomes
experiments

In [ ]:
# results_sobol_df = pd.DataFrame(results_sobol)
# results_sobol_df.to_excel("results_sobol_160s_10p.xlsx")
outcomes.keys()

In [ ]:
# one outcome for testing, do for all outcomes
y_welfare = np.mean(outcomes['Welfare'], axis =1) 

In [ ]:

sobol_indices = sobol.analyze(problem, y_welfare)
sobol_stats = {k:sobol_indices[k] for k in ['ST','ST_conf','S1','S1_conf']}
sobol_stats = pd.DataFrame(sobol_stats, index=problem['names'])
# starting with ST
sobol_stats.sort_values(by = 'ST', ascending=False) 

In [ ]:
# Change the outcomes to Z-scores by using the sobol analyze option
welfare_score = sobol.analyze(problem, y_welfare, calc_second_order=True, print_to_console=True)

In [ ]:
#  Visualize: The error bars indicate the confidence intervals.
sns.set_style('white')
fig, ax = plt.subplots(1)
indices = sobol_stats[['S1','ST']]
err = sobol_stats[['S1_conf','ST_conf']]

indices.plot.bar(yerr=err.values.T,ax=ax)
fig.set_size_inches(20,20)
fig.subplots_adjust(bottom=0.3)
plt.show()